In [1]:
import numpy as np
import scipy as sp

import skimage.io
import skimage.measure
import skimage.color

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

#Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['image.cmap'] = 'gray'
titlesize = 24
import sys, os

import SimpleITK as sitk
import nrrd


from scipy import ndimage
import pandas as pd
from radiomics import featureextractor
import joblib
from PIL import Image, ImageDraw
import cv2
import six

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import geojson
from PIL import Image, ImageDraw

import cv2
from json import JSONDecodeError

import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import (
    color, feature, filters, measure, morphology, segmentation, util)

In [48]:
import logging
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# ... or set level for specific class
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)

In [49]:
PATH = "/media/user/Seagate_RNA/PCNSL_PSL/1_patches_filiot_224/"
PATH_json = '/media/user/Seagate_RNA/PCNSL_PSL/3_hovernet_normalisation_reinhard_224/'
PATH_output_mask_nuclei = "/media/user/Seagate_RNA/PCNSL_PSL/4_masks_nuclei_pyradiomics//"
PATH_df_pyradiomics = "/media/user/Seagate_RNA/PCNSL_PSL/4_df_pyradiomics/"
PATH_df_count_cells = '/media/user/Seagate_RNA/PCNSL_PSL/4_df_count_cells/'

In [50]:
#on enlève les bords et on garde diametre entre 8 3.2 et 21
width = 224
marge = 1
res = 0.23

nbre_pix_inf = 32
nbre_pix_sup = 350

In [51]:
from PIL import Image, ImageDraw
def function_cell(cell) :
    width = 224
    poly_cell = []
    for x in cell["contour"] :
        poly_cell.append((x[0], x[1]))
    width = width
    height = width
    
    mask_cell = np.zeros((width, width), dtype = int)
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(poly_cell, outline = 1, fill=1)
    mask = np.array(img)
    mask_cell += mask
    
    mask_cell = np.array(mask_cell, dtype = 'uint8')

    
    return mask_cell

In [52]:
patients = os.listdir(PATH)
len(patients)

106

In [54]:
def nbre_pixels(cnt1):
    mask_cell = function_cell(cnt1)
    number_of_white_pix = np.sum(mask_cell != 0)
    return number_of_white_pix

In [56]:
list_excluded_keys = ['diagnostics_Versions_PyRadiomics',
 'diagnostics_Versions_Numpy',
 'diagnostics_Versions_SimpleITK',
 'diagnostics_Versions_PyWavelet',
 'diagnostics_Versions_Python',
 'diagnostics_Configuration_Settings',
 'diagnostics_Configuration_EnabledImageTypes',
 'diagnostics_Image-original_Hash',
 'diagnostics_Image-original_Dimensionality',
 'diagnostics_Image-original_Spacing',
 'diagnostics_Image-original_Size',
 'diagnostics_Image-original_Mean',
 'diagnostics_Image-original_Minimum',
 'diagnostics_Image-original_Maximum',
 'diagnostics_Mask-original_Hash',
 'diagnostics_Mask-original_Spacing',
 'diagnostics_Mask-original_Size',
 'diagnostics_Mask-original_BoundingBox',
 'diagnostics_Mask-original_VoxelNum',
 'diagnostics_Mask-original_VolumeNum',
 'diagnostics_Mask-original_CenterOfMassIndex',
 'diagnostics_Mask-original_CenterOfMass']

In [53]:
# list_excluded_keys = []
# for key, val in six.iteritems(result) :
#     if "diagnostics" in key :
#         list_excluded_keys.append(key)

In [1]:

count = 0
for patient in patients :
    print(patient)
    if not os.path.exists(PATH_df_pyradiomics+patient+".csv") :
        df = pd.DataFrame()
        df_counts = pd.read_csv(PATH_df_count_cells+patient+'.csv')
        patchs_file = []
        count_cells = 0
        for i in range(len(df_counts)) :
            if count_cells < 5000 :
                patchs_file.append(df_counts.iloc[i]['patch'])
                count_cells += df_counts.iloc[i]['cells_filtre_tot']

        patchs = []
        for i in patchs_file :
            patchs.append(i[:-4])
        print(len(patchs))
        for patch in patchs :
            input_image_file = np.asarray(Image.open(PATH+patient+"/"+patch+"jpg"))[:,:,0]
            with open(PATH_json+patient+'/json/'+patch+'json') as f :
                nuclei = geojson.load(f)

            key_keep = []
            key_not_keep_bords = []
            key_not_keep_diametre = []
            key_not_keep = []


            for key in list(nuclei['nuc'].keys()) :
                for x in nuclei['nuc'][key]['contour'] :
                    if x[0] < marge or x[0] > (width-marge) or x[1] < marge or x[1] > (width-marge) :
                        key_not_keep_bords.append(key)     
                    pix_count = nbre_pixels(nuclei['nuc'][key])

                    if pix_count < nbre_pix_inf or pix_count > nbre_pix_sup : #
                        key_not_keep_diametre.append(key)

                    key_not_keep_bords = list(set(key_not_keep_bords))
                    key_not_keep_diametre = list(set(key_not_keep_diametre))
                    key_not_keep = key_not_keep_bords + key_not_keep_diametre
                    key_not_keep = list(set(key_not_keep))

                for x in nuclei['nuc'][key]['contour'] :
                    if key not in key_not_keep and key not in key_keep :
                        key_keep.append(key) 

            for key in key_keep :
                #print(key)
                mask_cellmasks = function_mask_cellmasks_nuclei(nuclei['nuc'][key])
                if not os.path.exists(PATH_output_mask_nuclei+patient) :
                    os.makedirs(PATH_output_mask_nuclei+patient)
                nrrd.write(PATH_output_mask_nuclei+patient+"/"+patch+"__mask.nrrd", mask_cellmasks)
                nrrd.write(PATH_output_mask_nuclei+patient+"/"+patch+"__img.nrrd", input_image_file)
                settings = {'binWidth': 25,
                    'interpolator': sitk.sitkBSpline,
                    'resampledPixelSpacing': None,
                    'force2D': False,
                    'force2Ddimension': 0}
                imageType = {'Original': {}, 'Wavelet' : {}, 'LoG':None,'Square' : None, 'SquareRoot' : None, 'Logarithm' : None,'Exponential': None, 'Gradient': None,'LocalBinaryPattern2D':None, 'LocalBinaryPattern3D' : None}

             #   imageType = {'Original': {}, 'Wavelet' : {}, 'LoG':None,'Square' : None, 'SquareRoot' : None, 'Logarithm' : None,'Exponential': None, 'Gradient': None,'LocalBinaryPattern2D':None, 'LocalBinaryPattern3D' : None}

                extractor = featureextractor.RadiomicsFeatureExtractor() #**settings)#, **imageType)
                extractor.enableImageTypeByName('Wavelet')
                #extractor.enableImageTypeByName('LocalBinaryPattern2D')
               # extractor.enableFeatureClassByName('shape')
                extractor.enableFeatureClassByName('shape2D')

                try :
                    result = extractor.execute(PATH_output_mask_nuclei+patient+"/"+patch+"__img.nrrd",PATH_output_mask_nuclei+patient+"/"+patch+"__mask.nrrd")

                except RuntimeError :
                    print('erreur', patch)
                    continue
                features_names = {'ID_patient' : patient, 'patch' : patch, 'key' : key}

                for key, val in six.iteritems(result) :
                    if key in list_excluded_keys :
                        pass
                    else :
                        features_names[key] = val
                df_temp = pd.DataFrame.from_dict(data=features_names, orient='index')

                df = pd.concat([df, df_temp.T])
                #print(len(list(df.columns)))
                if len(df)%1000==0:
                    print('len df temp', len(df))
        print(len(df))
        df.to_csv(PATH_df_pyradiomics+patient+".csv")
        print('stocké')
    else:
        print('déja calculé')
    count += 1
    print('count', count)